In [1]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pandas as pd
from IPython.display import Markdown, display,HTML,Image
from selenium import webdriver
import time

In [2]:
# get the solution links
pages = ['https://www.janestreet.com/puzzles/archive/index.html',
         'https://www.janestreet.com/puzzles/archive/page2/index.html',
         'https://www.janestreet.com/puzzles/archive/page3/index.html',
         'https://www.janestreet.com/puzzles/archive/page4/index.html',
         'https://www.janestreet.com/puzzles/archive/page5/index.html',
         'https://www.janestreet.com/puzzles/archive/page6/index.html',
         'https://www.janestreet.com/puzzles/archive/page7/index.html',
         'https://www.janestreet.com/puzzles/archive/page8/index.html',
         'https://www.janestreet.com/puzzles/archive/page9/index.html',
         'https://www.janestreet.com/puzzles/archive/page10/index.html',
         'https://www.janestreet.com/puzzles/archive/page11/index.html',
         'https://www.janestreet.com/puzzles/archive/page12/index.html']  
links = ['https://www.janestreet.com/puzzles/current-puzzle/']
for url in pages:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    links += ['https://www.janestreet.com/'+i['href'] for i in soup.find_all('a', {'class' :'solution-link'})]   

In [3]:
# scrape the names of solvers
participants = []
order = 1
driver = webdriver.Firefox()
for url in links:
    driver.get(url)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    participants += [[str(url).split("/")[-1],i.split(" (")[0],order] for i in soup.find_all('p', {'class' :'correct-submissions'})[0].stripped_strings]
    order +=1

In [185]:
# Work out the scores with an offset. Count as attempted all those since the name appeared on a leaderboard.
# No adjustments for similar names or n
# Borrowed the offset idea from this (much nicer) tableau league table 
# https://public.tableau.com/app/profile/heidi.stockton/viz/PuzzlesofJaneStreet/JaneStreet

def df_format(styler):
    styler.format({'Score':'{:.2%}','Solved':'{:.0f}','Attempted':'{:.0f}'})
    styler.hide(axis=0,names=True)
    styler.set_table_styles([{'selector': 'td', 'props': [('text-align', 'center')]},
                             {'selector': '.col_heading', 'props': [('text-align', 'center'),
                                                                    ('width', '100px'),
                                                                    ('background-color', 'black'),
                                                                    ('color', 'white')]},
                             {'selector': '.row_heading', 'props': [('text-align', 'center'),
                                                                    ('background-color', 'black'),
                                                                    ('color', 'white')]}])
    return styler

newcomer_offset = 3

df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
df2.columns = [' '.join(col).strip() for col in df2.columns.values]
df2.columns = ['Solved','Attempted']
df2['Score']=df2['Solved']/(df2['Attempted']+ newcomer_offset)

df2.sort_values('Score',ascending=False)[:15].style.pipe(df_format)

,Solved,Attempted,Score
Evan Semet,26,26,89.66%
Karl Mahlburg,44,50,83.02%
Senthil Rajasekaran,49,61,76.56%
Gareth Owen,32,41,72.73%
Lazar Ilic,29,38,70.73%
Aaditya Raghavan,33,44,70.21%
Calvin Pozderac,52,74,67.53%
Arthur Bright,12,15,66.67%
Blaine Hill,6,6,66.67%
Sean Egan,47,68,66.20%


In [147]:
df2.sort_values('Solved',ascending=False)[:15].style.pipe(df_format)

,Solved,Attempted,Score
Calvin Pozderac,52,74,67.53%
Senthil Rajasekaran,49,61,76.56%
Sean Egan,47,68,66.20%
Karl Mahlburg,44,50,83.02%
Keith Schneider,41,65,60.29%
Heidi Stockton,39,85,44.32%
Aaditya Raghavan,33,44,70.21%
Sébastien Geeraert,32,59,51.61%
Gareth Owen,32,41,72.73%
Michael DeLyser,30,63,45.45%


In [145]:
print(df2.sort_values('Score',ascending=False)[:15].to_markdown())

| Name                |   Solved |   Attempted |    Score |
|:--------------------|---------:|------------:|---------:|
| Evan Semet          |       26 |          26 | 0.896552 |
| Karl Mahlburg       |       44 |          50 | 0.830189 |
| Senthil Rajasekaran |       49 |          61 | 0.765625 |
| Gareth Owen         |       32 |          41 | 0.727273 |
| Lazar Ilic          |       29 |          38 | 0.707317 |
| Aaditya Raghavan    |       33 |          44 | 0.702128 |
| Calvin Pozderac     |       52 |          74 | 0.675325 |
| Arthur Bright       |       12 |          15 | 0.666667 |
| Blaine Hill         |        6 |           6 | 0.666667 |
| Sean Egan           |       47 |          68 | 0.661972 |
| Cubist              |       30 |          43 | 0.652174 |
| Philip Bui          |        9 |          11 | 0.642857 |
| Danica Xiong        |        7 |           8 | 0.636364 |
| Keith Schneider     |       41 |          65 | 0.602941 |
| Will Owens          |        9 |      

In [183]:
# How does my position change with Newcomer Offset

for newcomer_offset in range(20):

    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempted']
    df2['Score']=df2['Solved']/(df2['Attempted']+ newcomer_offset)

    df3 = df2.sort_values('Score',ascending=False).reset_index() 
    print(newcomer_offset,df3[df3['Name'] =='Gareth Owen'].index[0]+1)

0 29
1 6
2 5
3 4
4 4
5 4
6 4
7 4
8 4
9 4
10 4
11 4
12 5
13 5
14 5
15 5
16 5
17 6
18 6
19 6
